In [ ]:
import os
import cv2
import numpy as np
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
import joblib
import multiprocessing
from tensorflow.keras.models import load_model

def load_images_from_directory(directory):
    images = []
    labels = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
                img_path = os.path.join(root, file)
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                if img is not None:
                    img = cv2.resize(img, (220, 220))
                    images.append(img.flatten())
                    label = os.path.basename(root)
                    labels.append(label)
    return np.array(images), np.array(labels)

# Load training and validation data
train_dir = './deepfake/train'
val_dir = './deepfake/val'

print("Loading training data...")
X_train, y_train = load_images_from_directory(train_dir)
print("Loading validation data...")
X_val, y_val = load_images_from_directory(val_dir)

# Encode labels
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_val_encoded = le.transform(y_val)

# Define base SVM model
base_svm = SVC(kernel='rbf', probability=True, random_state=42)

# Train Bagging model with SVM as base estimator using full CPU utilization
print("Training Bagging model with SVM base estimator using full CPU utilization...")
num_cores = multiprocessing.cpu_count()
bagging_svm_model = BaggingClassifier(base_estimator=base_svm, n_estimators=10, random_state=42, n_jobs=num_cores)
bagging_svm_model.fit(X_train, y_train_encoded)
print("Model trained successfully.")

# Cross-validation with full CPU utilization
print("Performing cross-validation...")
cv_scores = cross_val_score(bagging_svm_model, X_train, y_train_encoded, cv=5, scoring='accuracy', n_jobs=-1)
print(f"Cross-validation scores: {cv_scores}")
print(f"Mean CV Accuracy: {cv_scores.mean():.4f}")

# Evaluate the model
print("Evaluating the model...")
y_pred = bagging_svm_model.predict(X_val)

# Calculate evaluation metrics
conf_matrix = confusion_matrix(y_val_encoded, y_pred)
accuracy = accuracy_score(y_val_encoded, y_pred)
precision = precision_score(y_val_encoded, y_pred, average='weighted')
recall = recall_score(y_val_encoded, y_pred, average='weighted')
f1 = f1_score(y_val_encoded, y_pred, average='weighted')

print("Evaluation completed.")
print("Confusion Matrix:")
print(conf_matrix)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

# Save the trained model
print("Saving the trained model...")
joblib.dump(bagging_svm_model, 'bagging_svm_model_images.pkl')
print("Model saved successfully.")
